# State Management and Sessions

Implement session management with `FileSessionManager`. This notebook demonstrates how to maintain conversation context and handle multiple concurrent sessions for persistent agent interactions.

## What You'll Learn

- Implement conversation state persistence
- Manage multiple user sessions concurrently
- Handle context continuity across agent restarts
- Configure session storage and retrieval

## Prerequisites

- Completed [Notebook 03: MCP Integration](03-mcp-integration.ipynb)
- Understanding of file system operations
- Familiarity with conversation management concepts

📚 **Learn More**: [Session Management Documentation](https://strandsagents.com/latest/user-guide/concepts/agents/session-management/)

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel
from strands.session.file_session_manager import FileSessionManager
from strands_tools import image_reader, file_read
from video_reader_local import video_reader_local

print("✅ Imports successful!")

## Understanding Conversation History

By default, agents maintain conversation history **within a single execution**. This means:
- ✅ Context is preserved during the agent's lifetime
- ❌ Context is lost when the program ends
- ❌ No persistence across restarts

Let's see this in action:

📚 **Learn More**: [Conversation Management](https://strandsagents.com/latest/user-guide/concepts/agents/conversation-management/)

In [ ]:
# Setup Bedrock model
session = boto3.Session(region_name='us-west-2')
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# Create multi-modal agent (same as notebooks 02 and 03)
multimodal_agent = Agent(
    model=bedrock_model,
    tools=[image_reader, file_read, video_reader_local],
    system_prompt="""You are a multi-modal assistant that can:
    - Read and analyze images
    - Process documents (PDF, CSV, DOCX, etc.)
    - Analyze videos and provide detailed insights
    """
)

print("✅ Multi-modal agent created!")
print("🎨 Tools: image_reader, file_read, video_reader_local")

In [ ]:
# First interaction - analyze an image
response1 = multimodal_agent("Analyze the image at data-sample/diagram.jpg and tell me what you see.")
print("Response 1:", response1)
print("\n" + "="*80 + "\n")

# Second interaction - agent remembers the previous image analysis
response2 = multimodal_agent("What AWS services did you see in that image?")
print("Response 2:", response2)
print("\n💡 The agent remembered the image analysis from the previous message!")

## Conversation Managers: Managing Context Windows

Before diving into sessions, let's understand **Conversation Managers**.

### The Challenge

As conversations grow, we face:
- 🚫 **Token Limits**: Models have fixed context windows
- ⚡ **Performance**: Larger contexts = slower processing
- 📉 **Relevance**: Older messages may become less relevant

### Solution: Conversation Managers

Strands provides three built-in strategies:

1. **NullConversationManager**: No management (keeps all messages)
2. **SlidingWindowConversationManager**: Keeps N most recent messages (default)
3. **SummarizingConversationManager**: Summarizes old messages

📚 **Learn More**: [Conversation Management](https://strandsagents.com/latest/user-guide/concepts/agents/conversation-management/)

In [ ]:
from strands.agent.conversation_manager import SlidingWindowConversationManager

# Create multi-modal agent with sliding window (keeps last 10 messages)
windowed_agent = Agent(
    model=bedrock_model,
    tools=[image_reader, file_read, video_reader_local],
    conversation_manager=SlidingWindowConversationManager(
        window_size=10,  # Keep only 10 most recent messages
        should_truncate_results=True  # Truncate large tool results
    ),
    system_prompt="""You are a multi-modal assistant with sliding window memory that can:
    - Analyze images and videos
    - Process documents
    """
)

print("✅ Multi-modal agent with sliding window created!")
print("📊 Window size: 10 messages")

In [ ]:
# Simulate a long conversation with multi-modal content
print("Sending 15 messages to test sliding window...\n")

for i in range(15):
    if i == 0:
        windowed_agent("Analyze the image at data-sample/diagram.jpg")
    elif i == 5:
        windowed_agent("Read the document at data-sample/Welcome-Strands-Agents-SDK.pdf")
    else:
        windowed_agent(f"Message {i+1}: Tell me a fact about AI agents")

print(f"\n📊 Total messages sent: 15")
print(f"📊 Messages in memory: {len(windowed_agent.messages)}")
print(f"\n💡 Older messages (including the image and document analysis) were automatically removed!")
print(f"   Only the 10 most recent messages remain in the window.")

### Summarizing Conversation Manager

Instead of discarding old messages, we can **summarize** them:

In [ ]:
from strands.agent.conversation_manager import SummarizingConversationManager

# Create multi-modal agent with summarization
summarizing_agent = Agent(
    model=bedrock_model,
    tools=[image_reader, file_read, video_reader_local],
    conversation_manager=SummarizingConversationManager(
        summary_ratio=0.3,  # Summarize 30% of messages when needed
        preserve_recent_messages=5  # Always keep 5 most recent messages
    ),
    system_prompt="""You are a multi-modal assistant with summarization memory that can:
    - Analyze images and videos
    - Process documents
    Old conversations will be summarized to preserve important information.
    """
)

print("✅ Multi-modal agent with summarization created!")
print("📝 Old messages (including multi-modal content) will be summarized instead of discarded")

## Session Management: Persistence Across Executions

Conversation Managers handle **in-memory** context. For **persistent** conversations across executions, use **Session Managers**.

### What Gets Persisted?

Session Managers automatically save:
- 💬 **Conversation History**: All messages
- 🔑 **Agent State**: Key-value storage
- 📊 **Conversation Manager State**: Summaries, window state
- 🔄 **Multi-Agent State**: Orchestrator configuration (Graph/Swarm)

📚 **Learn More**: [Session Management](https://strandsagents.com/latest/user-guide/concepts/agents/session-management/)

In [ ]:
# Create a session manager
session_manager = FileSessionManager(
    session_id="multimodal-user-session",
    storage_dir="sessions"
)

# Create multi-modal agent with session manager
persistent_multimodal_agent = Agent(
    model=bedrock_model,
    tools=[image_reader, file_read, video_reader_local],
    session_manager=session_manager,
    system_prompt="""You are a multi-modal assistant with persistent memory that can:
    - Analyze images and videos
    - Process documents (PDF, CSV, DOCX, etc.)
    - Remember all interactions across sessions
    """
)

print("✅ Multi-modal agent with session manager created!")
print(f"📁 Session stored in: sessions/session_multimodal-user-session/")
print(f"🎨 Tools: image_reader, file_read, video_reader_local")

In [ ]:
# First conversation - analyze an image
print("=== 📸 First Interaction: Image Analysis ===")
response = persistent_multimodal_agent("Analyze the image at data-sample/diagram.jpg and describe the architecture.")
print(response)
print("\n💾 This interaction is now saved to the session!")

In [ ]:
# Continue conversation - agent remembers the image
print("\n=== 🔄 Second Interaction: Recall Previous Analysis ===")
response = persistent_multimodal_agent("What AWS services did you identify in that architecture diagram?")
print(response)
print("\n✅ The agent remembered the image analysis from the previous interaction!")

In [ ]:
# Add video analysis to the session
print("\n=== 🎥 Third Interaction: Video Analysis ===")
response = persistent_multimodal_agent("Now analyze the video at data-sample/climbing-video.mp4")
print(response)
print("\n💾 Video analysis also saved to session!")

## Session Storage Structure

FileSessionManager stores sessions in a structured format:

```
sessions/
└── session_user-alice-session/
    ├── session.json
    └── agents/
        └── agent_default/
            ├── agent.json
            └── messages/
                ├── message_0.json
                ├── message_1.json
                └── message_2.json
```

## Managing Multiple Sessions

You can create different sessions for different users or contexts:

In [ ]:
# Session for User 1 - Image analyst
session_user1 = FileSessionManager(
    session_id="image-analyst-session",
    storage_dir="sessions"
)

agent_user1 = Agent(
    model=bedrock_model,
    tools=[image_reader, file_read, video_reader_local],
    session_manager=session_user1,
    system_prompt="You are a multi-modal assistant specializing in image analysis."
)

# Session for User 2 - Video analyst
session_user2 = FileSessionManager(
    session_id="video-analyst-session",
    storage_dir="sessions"
)

agent_user2 = Agent(
    model=bedrock_model,
    tools=[image_reader, file_read, video_reader_local],
    session_manager=session_user2,
    system_prompt="You are a multi-modal assistant specializing in video analysis."
)

print("✅ Multiple multi-modal session agents created!")
print("👤 User 1: Image analyst")
print("👤 User 2: Video analyst")

In [ ]:
# User 1 conversation - Image analysis
print("=== 👤 User 1: Image Analyst ===")
response1 = agent_user1("Analyze the architecture diagram at data-sample/diagram.jpg")
print("User 1:", response1)
print("\n" + "="*80 + "\n")

# User 2 conversation - Video analysis
print("=== 👤 User 2: Video Analyst ===")
response2 = agent_user2("Analyze the video at data-sample/moderation-video.mp4")
print("User 2:", response2)
print("\n💡 Each user has their own independent session with multi-modal content!")

In [ ]:
# Verify that sessions are stored
import os
import json

print("\n" + "="*80)
print("=== 💾 Verifying that conversations are stored ===")
print("="*80 + "\n")

# Check User 1 session
user1_session_path = "sessions/session_image-analyst-session"
if os.path.exists(user1_session_path):
    print("✅ User 1 (Image Analyst) session found:")
    print(f"   📁 Path: {user1_session_path}")
    
    # Count messages for User 1
    user1_messages_dir = os.path.join(user1_session_path, "agents/agent_default/messages")
    if os.path.exists(user1_messages_dir):
        user1_message_count = len([f for f in os.listdir(user1_messages_dir) if f.endswith('.json')])
        print(f"   💬 Total messages stored: {user1_message_count}")
    print()
else:
    print("❌ User 1 session not found\n")

# Check User 2 session
user2_session_path = "sessions/session_video-analyst-session"
if os.path.exists(user2_session_path):
    print("✅ User 2 (Video Analyst) session found:")
    print(f"   📁 Path: {user2_session_path}")
    
    # Count messages for User 2
    user2_messages_dir = os.path.join(user2_session_path, "agents/agent_default/messages")
    if os.path.exists(user2_messages_dir):
        user2_message_count = len([f for f in os.listdir(user2_messages_dir) if f.endswith('.json')])
        print(f"   💬 Total messages stored: {user2_message_count}")

    print()
else:
    print("❌ User 2 session not found\n")


## Built-in Session Managers

Strands Agents provides two production-ready session managers:

### 1. FileSessionManager 📁
- **Storage**: Local filesystem
- **Use Case**: Development, testing, single-machine deployments
- **Pros**: Simple, fast, no external dependencies
- **Cons**: Not suitable for distributed systems

### 2. S3SessionManager ☁️
- **Storage**: Amazon S3
- **Use Case**: Production, distributed systems, cloud deployments
- **Pros**: Scalable, durable, accessible from anywhere
- **Cons**: Requires AWS credentials and S3 bucket

📚 **API Reference**: 
- [FileSessionManager](https://strandsagents.com/latest/api-reference/session/#strands.session.file_session_manager.FileSessionManager)
- [S3SessionManager](https://strandsagents.com/latest/api-reference/session/#strands.session.s3_session_manager.S3SessionManager)

In [ ]:
# Inspect session data
import json
import os

session_path = "sessions/session_multimodal-user-session/session.json"
if os.path.exists(session_path):
    with open(session_path, 'r') as f:
        session_data = json.load(f)
    print("📊 Session Data:")
    print(json.dumps(session_data, indent=2))
    
    # Count messages
    messages_dir = "sessions/session_multimodal-user-session/agents/agent_default/messages"
    if os.path.exists(messages_dir):
        message_count = len([f for f in os.listdir(messages_dir) if f.endswith('.json')])
        print(f"\n💬 Total messages persisted: {message_count}")
        print(f"📦 This includes multi-modal content (images, videos, documents)")
else:
    print("ℹ️ No session file found yet. Run the persistent agent cells first.")

## S3SessionManager: Cloud-Native Persistence

For production deployments, especially in distributed environments, use **S3SessionManager**.

### Benefits:
- ☁️ **Cloud-Native**: Fully managed by AWS
- 🌍 **Distributed**: Accessible from any AWS region
- 💪 **Durable**: 99.999999999% durability
- 📈 **Scalable**: Handles millions of sessions
- 🔒 **Secure**: IAM-based access control

### Required S3 Permissions:
```json
{
  "Version": "2012-10-17",
  "Statement": [{
    "Effect": "Allow",
    "Action": [
      "s3:PutObject",
      "s3:GetObject",
      "s3:DeleteObject",
      "s3:ListBucket"
    ],
    "Resource": [
      "arn:aws:s3:::my-sessions-bucket/*",
      "arn:aws:s3:::my-sessions-bucket"
    ]
  }]
}
```

📚 **Learn More**: [S3SessionManager Documentation](https://strandsagents.com/latest/user-guide/concepts/agents/session-management/#s3sessionmanager)

In [ ]:
from strands.session.s3_session_manager import S3SessionManager

# Configuration
BUCKET_NAME = "strands-agents-sessions"  # Change to your bucket
SESSION_PREFIX = "production/"  # Optional prefix for organization

# Create S3 session manager
s3_session_manager = S3SessionManager(
    session_id="user-bob-session",
    bucket=BUCKET_NAME,
    prefix=SESSION_PREFIX,
    boto_session=session,  # Reuse boto3 session
    region_name="us-west-2"
)

print("✅ S3 Session Manager created!")
print(f"📦 Bucket: {BUCKET_NAME}")
print(f"📁 Prefix: {SESSION_PREFIX}")


In [ ]:
# Create multi-modal agent with S3 session manager
s3_multimodal_agent = Agent(
    model=bedrock_model,
    tools=[image_reader, file_read, video_reader_local],
    session_manager=s3_session_manager,
    system_prompt="""You are a cloud-native multi-modal assistant with S3-backed persistence that can:
    - Analyze images and videos
    - Process documents
    - Scale to production workloads
    """
)

print("✅ Multi-modal agent with S3 session manager created!")

In [ ]:
# Test S3 session persistence with multi-modal content
print("=== 📸 Analyzing Image with S3 Persistence ===")
response = s3_multimodal_agent("Analyze the architecture in data-sample/diagram.jpg")
print("Response:", response)
print(f"📍 Location: s3://{BUCKET_NAME}/{SESSION_PREFIX}session_user-bob-session/")


### S3 Storage Structure

Sessions in S3 follow the same structure as FileSessionManager:

```
s3://my-sessions-bucket/
└── production/
    └── session_user-bob-session/
        ├── session.json
        └── agents/
            └── agent_default/
                ├── agent.json
                └── messages/
                    ├── message_0.json
                    ├── message_1.json
                    └── message_2.json
```

### When to Use S3SessionManager?

✅ **Use S3SessionManager when:**
- Deploying to AWS (Lambda, ECS, EC2)
- Building distributed systems
- Need high durability and availability
- Multiple services need access to sessions
- Scaling to many users

❌ **Use FileSessionManager when:**
- Local development and testing
- Single-machine deployments
- Prototyping
- No AWS infrastructure

## Benefits of Session Management

### 1. Continuity
Conversations can span multiple interactions and time periods

### 2. Personalization
Remember user preferences and context

### 3. Debugging
Inspect conversation history for troubleshooting

### 4. Analytics
Analyze conversation patterns and user behavior

### 5. Recovery
Resume conversations after interruptions

## Practical Example: Customer Support Bot

Let's create a customer support bot with session management:

In [ ]:
def create_multimodal_support_agent(customer_id: str):
    """Create a multi-modal customer support agent with persistent session."""
    session_manager = FileSessionManager(
        session_id=f"customer-{customer_id}",
        storage_dir="sessions"
    )
    
    return Agent(
        model=bedrock_model,
        tools=[image_reader, file_read, video_reader_local],
        session_manager=session_manager,
        system_prompt="""You are a multi-modal customer support agent that can:
        - Analyze screenshots of errors
        - Review log files and documents
        - Watch video recordings of issues
        Remember customer issues and provide consistent support.
        Be friendly and professional."""
    )

# Create multi-modal support agent for customer
support_agent = create_multimodal_support_agent("12345")

print("✅ Multi-modal customer support agent created!")
print("🎨 Can analyze: screenshots, documents, videos")

In [ ]:
# First support interaction - customer shares a screenshot
print("=== 📸 Customer Support: Screenshot Analysis ===")
response = support_agent("I'm seeing an error. Here's a screenshot: data-sample/diagram.jpg")
print("Support:", response[:300] + "...")
print("\n" + "="*80 + "\n")

# Follow-up - agent remembers the screenshot
print("=== 🔄 Follow-up: Remembering Previous Context ===")
response = support_agent("Can you explain what those AWS services in the screenshot are doing?")
print("Support:", response[:300] + "...")
print("\n✅ The agent remembered the screenshot analysis and provided context-aware support!")

## Comparison: FileSessionManager vs S3SessionManager

| Feature | FileSessionManager | S3SessionManager |
|---------|-------------------|------------------|
| **Storage** | Local filesystem | Amazon S3 |
| **Setup** | No configuration | Requires S3 bucket + IAM |
| **Performance** | Very fast | Fast (network latency) |
| **Durability** | Single machine | 99.999999999% |
| **Scalability** | Limited by disk | Unlimited |
| **Distributed** | ❌ No | ✅ Yes |
| **Cost** | Free | S3 storage + API costs |
| **Use Case** | Development | Production |

### Best Practices

1. **Unique Session IDs**: Use unique IDs per user/conversation
2. **Session Cleanup**: Implement TTL for old sessions
3. **Error Handling**: Handle storage failures gracefully
4. **Monitoring**: Track session sizes and access patterns
5. **Security**: Use IAM roles, not hardcoded credentials

In [ ]:
!pip install strands-agents strands-agents-tools boto3 -q

## Summary

In this notebook, you learned:

✅ **Conversation Managers**: Managing context windows (Sliding Window, Summarization)

✅ **Session Management**: Persisting state across executions

✅ **FileSessionManager**: Local file-based persistence

✅ **S3SessionManager**: Cloud-native S3-backed persistence

✅ **Multiple Sessions**: Managing different users/contexts

✅ **Storage Structures**: How sessions are organized

✅ **Best Practices**: Production-ready patterns

### What We've Achieved

Our agents can now:
- 💬 Remember conversations within a session
- 💾 Persist state across restarts
- 👥 Handle multiple users independently
- ☁️ Scale to production with S3

### But There's a Limitation...

Session managers are **great** for:
- ✅ Maintaining conversation history
- ✅ Remembering what was said in **this** session
- ✅ Resuming interrupted conversations

However, they **cannot**:
- ❌ Search across multiple sessions
- ❌ Find relevant information from past conversations
- ❌ Understand semantic relationships between sessions
- ❌ Learn from historical interactions

### The Question

**What if you want understanding BETWEEN sessions?**

### Next Steps

Continue to **notebook 05** to learn how to:
- Set up Amazon S3 Vectors for your agents
- Enable semantic search across all conversations
- Build agents with true long-term memory
- Combine session management with vector search

**Let's unlock cross-session understanding!** 🎯

📚 **Resources**:
- [Session Management Documentation](https://strandsagents.com/latest/user-guide/concepts/agents/session-management/)
- [Conversation Management](https://strandsagents.com/latest/user-guide/concepts/agents/conversation-management/)
- [API Reference: Session](https://strandsagents.com/latest/api-reference/session/)